## Pull data from a range of games and store into database

In [1]:
import statsapi
# from google.cloud import bigquery
from datetime import date, timedelta, datetime
import requests
import time
import sys
import os
import csv
import glob
from pprint import pprint
import sqlite3
import hashlib
import json
from collections import defaultdict

In [2]:
# File paths, date constants 
PLAYER_STATS_FOLDER = 'stats/player/'
TEAM_STATS_FOLDER = 'stats/team/'
TODAY = date.today()
YESTERDAY = TODAY - timedelta(days = 1)

week_ago = TODAY - timedelta(days = 7)
SUNDAY = TODAY - timedelta(days=TODAY.weekday())

In [3]:
#Set start/end dates
start_date = date(2025, 7,9)
end_date = start_date + timedelta(days = 0)
currentSeason = start_date.year

In [4]:
def player_stat_data(player_id, group=None, season=None, stats='season', gamePk=None):
    """
    Fetch player stats from the MLB Stats API.

    Args:
        player_id (int): The player ID.
        group (str): Stat group, e.g., 'hitting', 'pitching', or 'fielding'.
        season (int): The season year (required for 'season' stats).
        stats (str): Stat type. Either 'season' or 'gameLog'.
        gamePk (int): Optional. If provided with 'gameLog', filters to specific game splits.

    Returns:
        dict: A single stat split (for game or season), or None if not found.
    """
    url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats"
    
    params = {
        "stats": stats
    }
    
    if season:
        params["season"] = season
    if group:
        params["group"] = group
    if stats=="gameLog" and gamePk:
        params["gamePk"] = gamePk
        
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    if not data["stats"] or not data["stats"][0]["splits"]:
        return None

    splits = data["stats"][0]["splits"]

    if stats == "gameLog" and gamePk is not None:
        for split in splits:
            if split["game"]["gamePk"] == gamePk:
                return split
        return None  # GamePk not found

    # For 'season' or all game logs
    return splits[0] if splits else None


In [5]:
#Normalize name for pulling special fields from boxscore
def normalize_name(name_raw):
    if ',' in name_raw:
        last, first = [part.strip() for part in name_raw.split(',', 1)]
        return f"{first} {last}".strip()
    return name_raw

In [37]:
schedule = []
current_date = start_date
while current_date <= end_date:
    day_schedule = statsapi.schedule(
        start_date=str(current_date),
        end_date=str(current_date),
        sportId=1
    )
    
    if day_schedule:
        schedule.extend(day_schedule)
    current_date += timedelta(days=1)

games = []
weather = []

#Pull available game data from schedule
for game in schedule:
    
    # for game in date:
    gamePk = game['game_id']
    
    
    #Get weather
    boxscore = statsapi.get("game_boxscore", {'gamePk': gamePk})
    for item in boxscore['info']:
        if item['label'] == "Weather":
            weather = item['value']
        elif item['label'] == "Att":
            attendance = item['value']
        
    games.append({
        "gamePk": game['game_id'],
        "gametime": game['game_datetime'][11:],
        "gameDate": game['game_date'],
        "season": int(game['game_date'][:4]),
        "awayId": game['away_id'],
        "awayName": game['away_name'],
        "homeId": game['home_id'],
        "homeName": game['home_name'],
        "weather": weather,
        "attendance": attendance or None,
        "venue": game['venue_name'],
        "status": game['status'][0]
    })
print(f"Loaded {len(games)} games for {schedule[0]['game_date']}")

Loaded 15 games for 2025-07-09


In [7]:
teams = statsapi.get("teams", params = {"sportId":1})
raw_teams = teams['teams']
teams = []
for t in raw_teams:
    teams.append({
        'teamId': t['id'],
        'name': t['clubName'],
        'league': t['league']['name'],
        'venue': t['venue']['name'],
        'division': t['division']['name']
    })        
    

In [8]:
#Create hash PK for GAME_STATS table
def hash_game_stat(entity, stat, gamePk=None):
    key = f"{entity}:{stat}:{gamePk}" if gamePk is not None else f"{entity}:{stat}"
    return hashlib.sha256(key.encode('utf-8')).hexdigest()

game_stats = []

seen_players = set()
#Get team and player stats for games in games through boxscore endpoint
for g in games:
    gamePk = g['gamePk']
    gameDate = g['gametime'][:10]
    boxscore = statsapi.get("game_boxscore", {'gamePk': gamePk})
    for side in ["home", "away"]:
        team_stats = boxscore["teams"][side]["teamStats"]
        team_id = boxscore["teams"][side]['team']['id']
        # sys.exit()
        #Add team stats for games in schedule from boxscore
        for group in ['batting','pitching','fielding']:
            group_stats = team_stats.get(group, {})
            for stat, val in group_stats.items():
                game_stats.append({
                    "statsKey": hash_game_stat(team_id,stat,gamePk), 
                    "entity": team_id,
                    "scope": 'team',
                    "group": group,
                    "stat": stat, 
                    "val": val, 
                    "gamePk": gamePk,
                    "positionsPlayed": None
                })
            #print(f"Loaded {side} team data from game: {boxscore['teams']['away']['team']['abbreviation']} vs. {boxscore['teams']['home']['team']['abbreviation']}")
            
            #Player stats from boxscore 
            
            for player_key, player_data in boxscore["teams"][side]["players"].items():
                player_id = player_data["person"]["id"]  # ✅ This is the integer ID
                player_name = player_data["person"]["fullName"] #Thanks chat 
                position = player_data["position"]["abbreviation"]
                
                for group in ["batting", "fielding", "pitching"]:
                    group_stats = player_data["stats"].get(group, {})
                    if len(group_stats) == 0: #If player did not hit or pitch or field, no stats
                        continue
                    for stat, val in group_stats.items():
                        game_stats.append({
                            "statsKey": hash_game_stat(player_id,stat,gamePk),
                            "entity":player_id,
                            "scope": 'player',
                            "group":group,
                            "stat":stat, 
                            "val":val, 
                            "gamePk":gamePk,
                            "positionsPlayed": position
                        })
                #print(f"Loaded player data for {player_name} {player_id} in game: {boxscore['teams']['away']['team']['abbreviation']} vs. {boxscore['teams']['home']['team']['abbreviation']}")
                seen_players.add(player_id)
len_players = len(seen_players)

In [ ]:
#Create hash PK for season_stats table
def hash_szn_stat(entity, stat, group, season):
    #Concatenate 
    key = f"{entity}:{stat}:{season}:{group}"
    
    return hashlib.sha256(key.encode('utf-8')).hexdigest()


#Get player season stats + players

season_stats = []
players = []
seen_players = set()
for stats in game_stats:
    if stats['scope'] != 'player' or stats['entity'] in seen_players:
        continue
    else:
        playerId = stats['entity']
    # print(f"Pulled season stats for {len(seen_players)} out of {len_players / 2}")
    
    person = statsapi.get("person", params={"personId": playerId})
    person = person['people'][0]
    players.append({
        "playerId": person['id'],
        "fullName": person['fullName'],
        "pitchHand": person['pitchHand']['code'],
        "batSide": person['batSide']['code'],
        "primaryPosition": person['primaryPosition']['abbreviation'],
        "height": person['height'],
        "weight": person['weight'],
        'age': person['currentAge'],
        'jersey': person.get('primaryNumber', None)
    })
    
    for group in ["hitting", "pitching", "fielding"]:
        data = player_stat_data(playerId, currentSeason)
        if data is None:
            continue
        for stat, val in data['stat'].items():
            stat_key = hash_szn_stat(playerId, stat, group, currentSeason)
            season_stats.append({
                "statsKey": stat_key,
                "entity": playerId,
                "scope": "player",
                "group": group,
                "stat":stat, 
                "val":val,  
                "season":currentSeason,
                "updateDate":TODAY,
            })
    
        
    seen_players.add(playerId)
print(f"Got {currentSeason} season stats for {len(players)} players")
            

    
    

In [ ]:

#Team season stats


def team_stat_data(team_id, group, season):
    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats"
    params = {
        "season": season,
        "group": group,
        "stats": "season",
        "updateDate": TODAY
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()["stats"][0]["splits"][0]


for team in teams:
    teamId = team['teamId']
    teamName = team['name']
    for group in ["hitting","pitching","fielding"]:
    
        team_season_stats = team_stat_data(teamId, group, currentSeason)
        if 'stat' in team_stat_data(teamId, group, currentSeason):
            for stat, val in team_season_stats['stat'].items():         
                stat_key = hash_szn_stat(teamId,stat,group,currentSeason)
                season_stats.append({
                    "statsKey": stat_key,
                    "entity": teamId,
                    "scope": "team",
                    "group": group,
                    "stat": stat,
                    "val": val,
                    "season":currentSeason,
                    "updateDate": TODAY
                })
            print(f"Stored team {group} season stats for {teamName}")
                
            
            


In [33]:
def is_valid_pitch(event):
    event_type = event.get("type")
    is_pitch_type = event_type == 'pitch'
    return is_pitch_type

def pbp_hash_key(gamePk, index, atBatIndex=None):
    key = f"{gamePk}:{index}:{atBatIndex}" if atBatIndex is not None else f"{gamePk}:{index}"
    return hashlib.sha256(key.encode('utf-8')).hexdigest()
all_sequences = defaultdict(list)
at_bats = []
#Games (list of str)
for game in games:
    gamePk=game['gamePk']
    pbp = statsapi.get("game_playByPlay", params={"gamePk": gamePk})
    all_plays = pbp['allPlays']
    game_sequence = []
    for atbat in all_plays:
        sequence = []
        index = atbat['atBatIndex']
        batter=atbat["matchup"]["batter"]["id"]
        pitcher= atbat["matchup"]["pitcher"]["id"]
        atbat_key = pbp_hash_key(gamePk, index)
        at_bats.append({
            "atbatKey": atbat_key,
            "gamePk": gamePk,
            "gameIndex": index,
            'splits': f"{atbat['matchup']['splits']['batter']},{atbat['matchup']['splits']['pitcher']}",
            'menOnBase': atbat['matchup']['splits']['menOnBase'],
            'inning': atbat['about']['inning'],
            'result': atbat['result']['eventType'],
            'batterId': batter,
            'pitcherId': pitcher,
            "batterSide": atbat['matchup']['batSide']['code'],
            "pitcherHand": atbat['matchup']['pitchHand']['code']
        })
        
        # season_stats = player_stat_data(pitcher, group = "pitching", season = 2025)
        
        for event in atbat.get("playEvents", []):
            
            if not is_valid_pitch(event):
                continue
            else:
                # pprint(event.get("pitchData",{}).get('breaks',{}))
                # Extract relevant pitch info
                pitch_number = event.get("pitchNumber")
                count = event.get("count", {})
                pitch_key = pbp_hash_key(gamePk, index, pitch_number)
                # Append to context
                sequence.append({
                    "pitchKey": pitch_key,
                    "atbatKey": atbat_key,
                    "pitchNumber": pitch_number,
                    "balls": count.get("balls", 0),
                    "strikes": count.get("strikes", 0),
                    "outs": count.get("outs", 0),
                    "pitchResult": event.get("details", {}).get("description", {}),
                    "pitchType": event.get("details", {}).get("type", {}).get("description", ""),
                    "speed": event.get("pitchData", {}).get("startSpeed", None),
                    "typeConfidence":event.get("pitchData",{}).get('typeConfidence',{}),
                    "spinRate": event.get("pitchData",{}).get('breaks',{}).get('spinRate',0)
                })
        all_sequences[gamePk].append(sequence)

In [ ]:
print(all_sequences.keys())
for l in all_sequences[777182]:
    pprint(l)
    break

In [40]:
with sqlite3.connect('database/mlb_db.db', timeout = 10) as conn:
    cursor = conn.cursor()

    cursor.execute("DROP TABLE IF EXISTS GAMES;")
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS GAMES (
    gamePk INTEGER PRIMARY KEY,
    gametime TEXT,
    gameDate TEXT,
    season INTEGER, 
    awayId INTEGER,
    homeId INTEGER,
    weather TEXT,
    attendance TEXT,
    venue TEXT,
    status TEXT
    );
    """)
    
    cursor.executemany("""
    INSERT OR IGNORE INTO GAMES (gamePk, gametime, gameDate, season, awayId, homeId, weather, attendance, venue, status)
        VALUES (:gamePk, :gametime, :gameDate, :season, :awayId, :homeId, :weather, :attendance, :venue, :status)
        ;
    """, games)
    cursor.execute("DROP TABLE IF EXISTS TEAMS;")
    cursor.execute("""            
    CREATE TABLE IF NOT EXISTS TEAMS (
    teamId INTEGER PRIMARY KEY,
    teamName TEXT,
    league TEXT,
    venue TEXT,
    division TEXT
    );
    """)
    
    conn.commit()
    cursor.executemany("""
    INSERT OR IGNORE INTO TEAMS (teamId, teamName, league, venue, division) 
    VALUES (:teamId, :name, :league, :venue, :division)
    ;
    """, teams)
    
    cursor.execute("DROP TABLE IF EXISTS PLAYERS;")
    cursor.execute("""CREATE TABLE IF NOT EXISTS PLAYERS (
    playerId INTEGER PRIMARY KEY,
    fullName TEXT,
    pitchHand TEXT,
    batSide TEXT,
    primaryPosition TEXT,
    height TEXT,
    weight INTEGER,
    age INTEGER,
    jersey TEXT
    );""")
    conn.commit()

    cursor.executemany("""
    INSERT OR IGNORE INTO PLAYERS (playerId, fullName, batSide, pitchHand, primaryPosition, height, weight, age, jersey)
    VALUES (:playerId, :fullName, :pitchHand, :batSide, :primaryPosition, :height, :weight, :age, :jersey)
    """, players)
    conn.commit() 
    
    cursor.execute("DROP TABLE IF EXISTS GAME_STATS;")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS GAME_STATS (
    statsKey TEXT PRIMARY KEY,
    entity TEXT,
    scope BINARY, 
    "group" TEXT, --GROUP is SQL keyword
    stat TEXT,
    val TEXT,
    gamePk INTEGER, 
    positionsPlayed TEXT
    );
    """)
    
    cursor.executemany("""
    INSERT OR IGNORE INTO GAME_STATS (statsKey, entity, scope, "group", stat, val, gamePk, positionsPlayed)
    VALUES(:statsKey, :entity, :scope, :group, :stat, :val, :gamePk, :positionsPlayed)
    ;
    """, game_stats)
    conn.commit()
    cursor.execute("DROP TABLE IF EXISTS SEASON_STATS;")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS SEASON_STATS (
    statsKey TEXT PRIMARY KEY,
    entity TEXT,
    scope TEXT,
    "group" TEXT,
    stat TEXT,
    val TEXT,
    season INTEGER,
    updateDate DATETIME
    );
    """)
    conn.commit()

    cursor.executemany("""
    INSERT OR IGNORE INTO SEASON_STATS (statsKey, entity, scope, "group", stat, val, season, updateDate)
    VALUES(:statsKey, :entity, :scope, :group, :stat, :val, :season, :updateDate)
    ;
    """, season_stats)
    conn.commit()

    
    cursor.execute("DROP TABLE IF EXISTS AT_BATS;")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS AT_BATS (
    atbatKey TEXT PRIMARY KEY,
    gamePk INTEGER,
    gameIndex INTEGER,
    splits TEXT,
    menOnBase TEXT,
    inning INTEGER,
    result TEXT,
    batterId INTEGER,
    pitcherId INTEGER,
    batterSide TEXT,
    pitcherHand TEXT
    );
    """)
    conn.commit()

    cursor.executemany("""
    INSERT OR IGNORE INTO AT_BATS (atbatKey, gamePk, gameIndex, splits, menOnBase, inning, result, batterId, pitcherId, batterSide, pitcherHand)
    VALUES(:atbatKey, :gamePk, :gameIndex, :splits, :menOnBase, :inning, :result, :batterId, :pitcherId, :batterSide, :pitcherHand)
    ;
    """, at_bats)

        
    cursor.execute("DROP TABLE IF EXISTS AT_BAT_SEQUENCE;")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS AT_BAT_SEQUENCE (
    pitchKey TEXT PRIMARY KEY,
    atbatKey TEXT,
    pitchNumber INTEGER,
    balls INTEGER,
    strikes INTEGER,
    outs INTEGER,
    pitchResult TEXT,
    pitchType TEXT,
    speed REAL,
    typeConfidence REAL,
    spinRate REAL,
    spinDirection INTEGER
    );
    """)
    for game in all_sequences.keys(): 
        seq = all_sequences.get(game,{}) #list of dicts
        for pitch in seq:
            cursor.executemany("""
            INSERT OR IGNORE INTO AT_BAT_SEQUENCE (pitchKey, atbatKey, pitchNumber, balls, strikes, outs, pitchResult, pitchType, speed, typeConfidence, spinRate)
            VALUES (:pitchKey, :atbatKey, :pitchNumber, :balls, :strikes, :outs, :pitchResult, :pitchType, :speed, :typeConfidence, :spinRate)
            """, pitch)
    
    conn.commit()

In [29]:
#Test query
cursor.execute("""SELECT DISTINCT games.gamePk, away.teamName, home.teamName
FROM games
INNER JOIN teams away
    ON games.awayId = away.teamId
INNER JOIN teams home
    ON games.homeId = home.teamId
INNER JOIN 
""")
for i in cursor.fetchall():
    print(f"{i[0]}: {i[2]} at {i[1]}")

777162: Padres at Diamondbacks
777163: Giants at Phillies
777171: Angels at Rangers
777172: White Sox at Blue Jays
777173: Twins at Cubs
777174: Astros at Guardians
777175: Brewers at Dodgers
777176: Athletics at Braves
777177: Royals at Pirates
777178: Cardinals at Nationals
777179: Red Sox at Rockies
777181: Orioles at Mets
777182: Tigers at Rays
777183: Reds at Marlins
777184: Yankees at Mariners
